<a href="https://colab.research.google.com/github/manashpratim/Bosch-Summer-Internship/blob/master/Text_Attack_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Twitter Sentiment Classification**

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Downloading the Datasets (https://www.kaggle.com/kazanova/sentiment140)
!wget --no-check-certificate \
      "https://storage.googleapis.com/kaggle-data-sets/2477%2F4140%2Fbundle%2Farchive.zip?GoogleAccessId=gcp-kaggle-com@kaggle-161607.iam.gserviceaccount.com&Expires=1595795502&Signature=RL%2B0kDJFHl%2Be5wEpXrHiJdkJ7FXXqIIiO%2B4LAzsK59KjW89xILFBwgCRn2M1MCiNPKiYTB469Y%2FeR17sXFlyF4sX6wB4bidNdqdLr9p%2F38MQiHRdbU%2BYpWa4R0pCmwYIzS%2BySdVAYh8dA8HhDKwo3STpaeLjENdtw%2BXtFcOfuKHOZx08YRi1UbOk5NxE15DA1c%2FTJDUF5lZype4okBqiMmr7x3qoHjBrQzcC7n60%2F1ZTmf14Htov%2BnXa6qdEBcfSoDfmBXrruegwBdFodSDkeVulE%2FirVN%2BOK%2BCVVYFEqJ85XOLQd64TtxZJmnxD%2BKuJRcPMVCn0s72bMmbRBejdsA%3D%3D"\
      -O "/content/sentiment_tweets.zip"

In [ ]:
!unzip -q '/content/sentiment_tweets.zip'

In [ ]:
import numpy as np
import pandas as pd
df=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding="ISO-8859-1",names = ["label", "ids", "date", "flag", "user", "tweets"])

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.bar([0,1], df['label'].value_counts(), width=0.5, bottom=None, align='center', data=df)
plt.title('Labels Distribution')
plt.xlabel('Labels')
plt.ylabel('Frequency')
plt.xticks(np.arange(0, 1, step=2))
print('Labels Distribution:\n',df['label'].value_counts())

In [ ]:
#Functions for Preprocessing the Dataset
import nltk
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
nltk.download('punkt')
stop_words = set(stopwords.words('english'))
from textblob import Word
nltk.download('wordnet')

def mystopwords(text):
    return ' '.join([w for w in word_tokenize(text) if not w in stop_words])

def lemmatize(text):
    return ' '.join([Word(word).lemmatize() for word in text.split()])

import re
def preprocess(text):
    # Remove link,user and special characters
    text = re.sub("@\S+|https?:\S+|http?:\S|[^A-Za-z0-9]+", ' ', str(text).lower()).strip()
    tokens = []
    for token in text.split():
          tokens.append(token)
    return " ".join(tokens)

In [ ]:
#Cleaning the train data and separating the texts and labels
tweets=[]
labels=[]
for i in range(len(df)):
  if df['label'][i]==4:
    labels.append(1)
  else:
    labels.append(0)
  tweets.append(preprocess(df['tweets'][i]))

In [ ]:
#Example of a tweet before and after cleaning
print('Tweet before cleaning: ',df['tweets'][100])
print('Tweet after cleaning: ',tweets[100])

In [ ]:
#Suffling the data and splitting the data into train and test sets (80:20)
from sklearn.model_selection import train_test_split
train_tweets, val_tweets, train_labels, val_labels= train_test_split(tweets, labels, test_size=0.2, shuffle= True,random_state=42)
#test_tweets = val_tweets[:250]
#test_labels = val_labels[:250]
val_tweets, test_tweets, val_labels, test_labels= train_test_split(val_tweets, val_labels, test_size=0.001, shuffle= True,random_state=42)

In [ ]:
#Data Summary
print('Length of the Training Set: ',len(df))
print('No. of Training Examples: ',len(train_tweets))
print('No. of Validation Examples: ',len(val_tweets))
print('No. of Test Examples: ',len(test_tweets))

In [ ]:
# Preprocess the text data. Similar to the audio data, segments of the text data are paddded to have same length
import tensorflow as tf
import numpy as np

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 40
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
vocab_len=50000

tokenizer = Tokenizer(num_words=vocab_len+1,oov_token=oov_tok)
tokenizer.fit_on_texts(tweets)

word_index = tokenizer.word_index
vocab_size=len(word_index)
print('Size of Vocabulary: ',vocab_size)

train_sequences = tokenizer.texts_to_sequences(train_tweets)
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

val_sequences = tokenizer.texts_to_sequences(val_tweets)
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_tweets)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

train_labels=np.expand_dims(train_labels, axis=1)
val_labels=np.expand_dims(val_labels, axis=1)
test_labels=np.expand_dims(test_labels, axis=1)

In [ ]:
# Data Statistics
print('Dimension of Training Data: ',train_padded.shape)
print('Dimension of Validation Data: ',val_padded.shape)
print('Dimension of Test Data: ',test_padded.shape)
print('Dimension of Training Labels: ',train_labels.shape)
print('Dimension of Validation Labels: ',val_labels.shape)
print('Dimension of Test Labels: ',test_labels.shape)

In [ ]:
# Download Glove Wiki Embeddings
!wget --no-check-certificate \
      "http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip"\
      -O "/content/drive/My Drive/mosi_data/globe6B.zip"

In [ ]:
# Unzip the downloaded embeddings
!unzip -q '/content/drive/My Drive/mosi_data/globe6B.zip'

In [ ]:
# Load the embeddings. There are 4 dimensions to choose from. I used 300 dimensional embeddings. 
embeddings_index = {}
with open('/content/glove.6B.300d.txt') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [ ]:
# Map the embeddings with the words of the text data
embedding_dim = 300
embeddings_matrix = np.zeros((vocab_size+1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector

#**Training** 

In [ ]:
# Function to compute F1 score. I use it as a metrics for Binary Classification.
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
# Provide the suitable units inside the Dense layer.
# For Binary classification, use 1 and 'sigmoid' as activation
# For 7 class classification, use 7 and 'softmax' as activation
# For Regression, use 1 and remove activation

from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

# text only model
inp = Input(max_length)   
layer = tf.keras.layers.Embedding(vocab_size+1, embedding_dim, input_length=max_length, weights = [embeddings_matrix], trainable = False)(inp)            
layer = tf.keras.layers.Dropout(0.4)(layer)
layer = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(128,return_sequences=True))(layer)
layer = tf.keras.layers.Bidirectional(tf.compat.v1.keras.layers.CuDNNLSTM(128))(layer)
layer = tf.keras.layers.Dropout(0.2)(layer)
layer = tf.keras.layers.Dense(128, activation='relu')(layer)
layer = tf.keras.layers.Dropout(0.4)(layer)
layer = tf.keras.layers.Dense(64, activation='relu')(layer)
layer = tf.keras.layers.Dropout(0.5)(layer)
#out = tf.keras.layers.Dense(1, activation='sigmoid')(layer) 
out = tf.keras.layers.Dense(2, activation='softmax')(layer) 
#out = tf.keras.layers.Dense(1)(layer)                 
model= Model(inp,out)                                 



In [ ]:
reduce =tf. keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, mode='auto')  #to reduce learning rate by factor of 0.1 if model performance degrades for 10 (patience) epochs.  
earlystopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True,monitor='val_accuracy', min_delta=1e-4,mode='max')  #early stopping if performance of model degrades for 10 epochs

#Uncomment one of the next three lines at a time
#model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy',f1_score])                #Binary classification
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])            #7 class classifiaction
#model.compile(loss="mean_absolute_error",optimizer='adam',metrics=["mean_absolute_error"])             #Regression

# I am training for 50 epochs with a batch size of 256. Set verbose to 2 for no training details and 0 for more training details.
history = model.fit(train_padded, train_labels, batch_size=4096, epochs=5, validation_data=(val_padded,val_labels),shuffle=True,callbacks=[reduce,earlystopping])

In [ ]:
#Test Accuracy
results = model.evaluate(x=test_padded,y=test_labels)
print('Test Set Performance: ',results)

## **Attack**

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
stop_words = set(stopwords.words('english'))
from absl import logging
from tqdm import tqdm
import tensorflow as tf

import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns
import sklearn
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
mode = hub.load(module_url)
print ("module %s loaded" % module_url)

def embed(input):
  return mode(input)

In [ ]:
from gensim.models import KeyedVectors
#model1 = KeyedVectors.load_word2vec_format('/content/drive/My Drive/GoogleNews-vectors-negative300.bin.gz (Unzipped Files)/GoogleNews-vectors-negative300.bin', binary=True)
!unzip -q '/content/drive/My Drive/mosi_data/fasttext_wiki.zip'
similarity_model = KeyedVectors.load_word2vec_format('/content/wiki-news-300d-1M.vec', binary=False)

In [ ]:
def evaluate_word_saliency(model,val_reviews,val_padded,val_labels):
    candidates = {}
    outer = tqdm(total=len(val_reviews), desc='Evaluating Word Saliency', position=0)
    for j,sent in enumerate(val_reviews):
        outer.update(1)
        sent = sent.split()
        score = model.predict(val_padded[j].reshape(1,-1))[0]
        pred = np.argmax(score)

        if pred==val_labels[j]:
          mask = np.zeros(40)
          mask[:len(sent)]=1
          dic = {}
          for i,word in enumerate(sent):
            if word not in stop_words:
              mask[i] = 0
              inp = mask*val_padded[j]
              score_new = model.predict(inp.reshape(1,-1))[0]
              pred_new = np.argmax(score_new)
              if pred_new == pred:
                imp = score[pred] - score_new[pred_new]
              else:
                  imp = (score[pred] - score_new[pred]) + (score_new[pred_new] - score[pred_new])
              mask[i] = 1
              if imp>0:
                dic[word] = imp
          if dic:
            k = sorted(dic.keys(),key = dic.get,reverse=True)
            if len(k)>3:
              candidates[j] = k[:3]
            else:
              candidates[j] = k

    return candidates

In [ ]:
def find_similar_words(model,val_reviews,val_padded,val_labels,candidates,model1,embed):
      
      new_candidates = {}

      outer = tqdm(total=len(candidates), desc='Finding Similar Words', position=0)
      for j,sent in enumerate(val_reviews):
        if j in candidates:
            outer.update(1)
            sent = sent.split()
            dic = {}
            score = model.predict(val_padded[j].reshape(1,-1))[0]
            pred = np.argmax(score)
            for i,word in enumerate(sent):
              if word in candidates[j]:
                n = []
                try:
                  l = model1.most_similar(word,topn=50)
                except:
                  pass

                mini = float('inf')
                we = None 
                for w in l:
                  if w[0].lower()!=word:
                    arr = sent[:]
                    arr[i] = w[0].lower()
                    k = tokenizer.texts_to_sequences([" ".join(arr)])
                    inp = pad_sequences(k, maxlen=40, padding='post', truncating='post')
                    score_new = model.predict(inp)[0]
                    pred_new = np.argmax(score_new)
                    if pred_new !=pred:
                      message_embeddings = embed([" ".join(arr)," ".join(sent)])
                      score1 =sklearn.metrics.pairwise.cosine_similarity(np.array(message_embeddings[0]).reshape(1,-1),np.array(message_embeddings[1]).reshape(1,-1))[0][0]
                      if score1>0.7:
                          n.append(w[0].lower())
                    else:
                      if score_new[pred]<mini:
                            mini = score_new[pred]
                            we = w[0].lower()
                if we:
                  arr = sent[:]
                  arr[i] = we
                  message_embeddings = embed([" ".join(arr)," ".join(sent)])
                  score1 =sklearn.metrics.pairwise.cosine_similarity(np.array(message_embeddings[0]).reshape(1,-1),np.array(message_embeddings[1]).reshape(1,-1))[0][0]
                  if score1>0.7:
                    n.append(we)
                if n:
                  dic[word] = n[:]
            if dic:
                new_candidates[j] = dic 

      return new_candidates

In [ ]:
def generate_adversary(model,val_reviews,val_padded,val_labels,new_candidates,embed):
    adversary = {}

    outer = tqdm(total=len(new_candidates), desc='Generating Adversarial Samples', position=0)
    for j,sent in enumerate(val_reviews):
      if j in new_candidates:
        outer.update(1)
        sent = sent.split()
        score = model.predict(val_padded[j].reshape(1,-1))[0]
        pred = np.argmax(score)
        a = []
        arr = sent[:]
        flag = 0
        for i,word in enumerate(sent):
          if word in new_candidates[j]:
            
            mini = float('inf')
            we = None
            
            for w in new_candidates[j][word]:
              if w:
                s = arr[i]
                arr[i] = w.lower()
                k = tokenizer.texts_to_sequences([" ".join(arr)])
                inp = pad_sequences(k, maxlen=40, padding='post', truncating='post')
                score_new = model.predict(inp)[0]
                pred_new = np.argmax(score_new)
                if pred_new !=pred:
                  arr1 = sent[:]
                  arr1[i] = w.lower()
                  message_embeddings = embed([" ".join(arr1)," ".join(sent)])
                  score1 =sklearn.metrics.pairwise.cosine_similarity(np.array(message_embeddings[0]).reshape(1,-1),np.array(message_embeddings[1]).reshape(1,-1))[0][0]
                  if score1>0.7:
                      arr = sent[:]
                      arr[i] = w.lower()
                      flag=1
                      break
                else:
                    
                    if  abs(score[pred] - score_new[pred])>0.4:
                        arr1 = sent[:]
                        arr1[i] = w.lower()
                        message_embeddings = embed([" ".join(arr1)," ".join(sent)])
                        score1 =sklearn.metrics.pairwise.cosine_similarity(np.array(message_embeddings[0]).reshape(1,-1),np.array(message_embeddings[1]).reshape(1,-1))[0][0]
                        if score1>0.7:
                              arr = sent[:]
                              arr[i] = w.lower()
                              flag = 1
                              break
                    else:
                      arr[i] = s
                      if score_new[pred]<mini:
                          mini = score_new[pred]
                          we = w.lower()
            if flag:
              break
            if we:
              arr[i] = we
        adversary[j] = " ".join(arr)

    adver = []

    for i in range(len(val_reviews)):
      if i in adversary:
        adver.append(adversary[i])
      else:
        adver.append(val_reviews[i])

    return adver

In [ ]:
candidates = evaluate_word_saliency(model,test_tweets,test_padded,test_labels)
new_candidates = find_similar_words(model,test_tweets,test_padded,test_labels,candidates,similarity_model,embed)
adversary = generate_adversary(model,test_tweets,test_padded,test_labels,new_candidates,embed)

In [ ]:
test_seq = tokenizer.texts_to_sequences(adversary)
test_pad = pad_sequences(test_seq, maxlen=40, padding='post', truncating='post')

In [ ]:
results1 = model.evaluate(x=test_padded,y=test_labels)
print('Test Set Performance: ',results1)
results2 = model.evaluate(x=test_pad,y=test_labels)
print('Test Set Performance after Attack: ',results2)

In [ ]:
new_candidates.keys()

In [ ]:
index = 50
print('Adversary: ',adversary[index])
k = tokenizer.texts_to_sequences([adversary[index]])
k = pad_sequences(k, maxlen=40, padding='post', truncating='post')
results = model.predict(x=k)
print('Adversary Prediction: ',np.argmax(results))

print('Original: ',test_tweets[index])
k1 = tokenizer.texts_to_sequences([test_tweets[index]])
k1 = pad_sequences(k1, maxlen=40, padding='post', truncating='post')
results2 = model.predict(x=k1)
print('Original Prediction: ',np.argmax(results2))